# Gen AI for BQML
This Notebook shows you how to create a BigQuery ML remote model that references a Vertex AI natural language foundation model. You can then use that model in conjunction with the ML.GENERATE_TEXT function to analyze text in a BigQuery table.

You can run any of the queries shown here within the Big Query Studio UI **accessible from the link from the cell below or through the cloud console**.

### Pre-Requisites
The Generative AI Learning Environment has already been setup with the required connector for Gen AI for BQML and it will be used in the following example code.

Import the Big Query Python SDK

This cell also generates a direct link to Big Query Studio within this project.

In [ ]:
# Cloud project id.
class format:
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    Default = '\033[0m'

PROJECT_ID = !gcloud config get project
PROJECT_ID = PROJECT_ID.n
print("Project ID: " + PROJECT_ID + "\n")

print(format.BOLD + "Biq Query UI Link: " + format.UNDERLINE + "https://console.cloud.google.com/bigquery?project={}".format(PROJECT_ID) + format.Default) 

from google.cloud import bigquery

### Create a BQML Model

This function creates a remote model using the pre-provisioned remote model connection within this environment, you can set the endpoint field to be any supported generative AI foundation model such as "code-bison@001").

Reference for Syntax: https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-create

In [ ]:
client = bigquery.Client()

create_endpoint_connection = """
    CREATE OR REPLACE MODEL `{}.gen_ai_dataset.textbison-model` REMOTE WITH CONNECTION `{}.europe-west2.bqllm` OPTIONS (ENDPOINT = 'text-bison@001')
""".format(PROJECT_ID,PROJECT_ID)
model_create_job = client.query(create_endpoint_connection)  # Make an API request.

### Simple Generation Query

The pre-loaded gen_ai_bqtable contains a list of prompts, the following example generates responses based on those prompts. As mentioned above you can run the query string directly within big-query studio for further experimentation (please make sure to replace the "{}" template within the string to the project id of your playpen).

In [ ]:
generate_using_prompts = """
SELECT *
FROM
  ML.GENERATE_TEXT(
    MODEL `{}.gen_ai_dataset.textbison-model`,
    (
    SELECT
        CONCAT(string_field_0)
        AS prompt from `{}.gen_ai_dataset.gen_ai_bqtable`
          limit 5
    ),
    STRUCT(
      0.4 AS temperature, 100 AS max_output_tokens, 0.5 AS top_p,
      40 AS top_k, TRUE AS flatten_json_output));
""".format(PROJECT_ID,PROJECT_ID)

model_generate_job_simple = client.query(generate_using_prompts)  # Make an API request.

print("Categories: \n")
for row in model_generate_job_simple:
    # Row values can be accessed by field name or index.
    print("prompt={} \n category={} \n\n\n".format(row["prompt"], row["ml_generate_text_llm_result"]))

### Further Example of a Generation Query
Gen AI for BQML can also be used for more complex data analysis tasks such as in the following example. This query uses the same data set of prompts as above and categorisies each sentence into a one word category. This can be used to quickly analyse datasets, for example if you want to extract the programming language of each row of a dataset containing code or for NLP tasks on large datasets such as sentiment analysis.

In [ ]:
categorise_prompts = """
SELECT
  ml_generate_text_result['predictions'][0]['content'] AS generated_text,
  ml_generate_text_result['predictions'][0]['safetyAttributes']
    AS safety_attributes,
  * EXCEPT (ml_generate_text_result)
FROM
  ML.GENERATE_TEXT(
    MODEL `{}.gen_ai_dataset.textbison-model`,
    (
    SELECT
        CONCAT('Could you categorise the following generative ai model prompt using only 1 word: ', string_field_0)
        AS prompt from `{}.gen_ai_dataset.gen_ai_bqtable`
          limit 5
    ),
    STRUCT(
      0.2 AS temperature,
      100 AS max_output_tokens))
""".format(PROJECT_ID,PROJECT_ID)

model_generate_job = client.query(categorise_prompts)

print("Categories: \n")
for row in model_generate_job:
    # Row values can be accessed by field name or index.
    print("prompt={} \t category={} \n".format(row["prompt"], row["generated_text"]))